In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
import joblib
import warnings

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data / 255.0, mnist.target.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("MNIST dataset loaded and preprocessed.")

MNIST dataset loaded and preprocessed.


In [3]:
svc = SVC()

In [4]:
param_grid = {
    'C': [1, 10, 100], 
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 0.01, 0.1],
    'shrinking': [True],
    'decision_function_shape': ['ovr']
}
'''
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(svc, param_distributions=param_grid, n_iter=30, cv=2, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
'''

random_search= joblib.load('random_search.pkl')
best_model= joblib.load('best_model.pkl')

c:\Users\manan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\manan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomizedSearchCV from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
#joblib.dump(random_search, 'random_search.pkl')
#joblib.dump(best_model, 'best_model.pkl')

In [6]:
print("Best hyperparameters found:")
print(random_search.best_params_)

Best hyperparameters found:
{'shrinking': True, 'kernel': 'rbf', 'gamma': 'scale', 'decision_function_shape': 'ovr', 'C': 10}


In [7]:
#y_pred = best_model.predict(X_test)

y_pred= np.load('y_pred.npy')

In [8]:
#np.save('y_pred.npy', y_pred)

In [9]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[1329    1    4    0    1    0    3    1    4    0]
 [   0 1588    3    3    1    0    0    5    0    0]
 [   2    3 1358    1    2    1    2    5    5    1]
 [   1    1   12 1401    2    6    0    5    2    3]
 [   1    0    2    0 1273    0    2    4    1   12]
 [   0    1    0   10    2 1247    8    0    5    0]
 [   1    0    0    0    5    4 1385    0    1    0]
 [   1    2   10    0    5    1    0 1478    1    5]
 [   3    3    4    8    1    6    4    4 1317    7]
 [   7    4    1    7    7    2    0   12    4 1376]]


In [10]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1343
           1       0.99      0.99      0.99      1600
           2       0.97      0.98      0.98      1380
           3       0.98      0.98      0.98      1433
           4       0.98      0.98      0.98      1295
           5       0.98      0.98      0.98      1273
           6       0.99      0.99      0.99      1396
           7       0.98      0.98      0.98      1503
           8       0.98      0.97      0.98      1357
           9       0.98      0.97      0.97      1420

    accuracy                           0.98     14000
   macro avg       0.98      0.98      0.98     14000
weighted avg       0.98      0.98      0.98     14000



In [11]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    resized = cv2.resize(binary, (28, 28), interpolation=cv2.INTER_AREA)
    reshaped = resized.reshape(1, -1)
    return reshaped / 255.0

In [12]:
url= 'http://192.168.0.123:8080/video'
cap = cv2.VideoCapture(url)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Webcam opened successfully.")

x_start, y_start = 625,300
width, height = 196, 196

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break
    cv2.rectangle(frame, (x_start, y_start), (x_start + width, y_start + height), (0, 255, 0), 2)
    roi = frame[y_start:y_start + height, x_start:x_start + width]
    preprocessed_frame = preprocess_image(roi)
    
    cv2.imshow('Captured Region', roi)
    cv2.imshow('Preprocessed Region', preprocessed_frame.reshape(28, 28))


    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UserWarning)
        prediction = best_model.predict(preprocessed_frame)

    #cv2.putText(frame, f"NUMBER: {prediction[0]}", (50,50), cv2.FONT_HERSHEY_TRIPLEX, 2, (151,213,179), 1)
    text = f"NUMBER: {prediction[0]}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    
    text_x = 50
    text_y = 100  # Adjusted for better positioning
    text_w = text_size[0] + 20
    text_h = text_size[1] + 20

    cv2.rectangle(frame, (text_x - 10, text_y - text_h - 10), (text_x + text_w, text_y + 10), (0, 0, 0), -1)
    cv2.putText(frame, text, (text_x, text_y), font, font_scale, (145,159,212), font_thickness, cv2.LINE_AA)


    cv2.imshow('Number Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Webcam opened successfully.
